In [1]:
import logging
import time

import awkward as ak
import cabinetry
import cloudpickle
import correctionlib
from coffea import processor
from coffea.nanoevents import NanoAODSchema
from coffea.analysis_tools import PackedSelection
import copy
import hist
import matplotlib.pyplot as plt
import numpy as np
import pyhf

import utils 

logging.getLogger("cabinetry").setLevel(logging.INFO)

In [51]:
N_FILES_MAX_PER_SAMPLE = 10
USE_DASK=False
USE_SERVICEX=False

In [52]:
fileset = utils.file_input.construct_fileset(
    N_FILES_MAX_PER_SAMPLE,
    use_xcache=False,
    af_name="coffea_casa",  
    input_from_eos=False,
    xcache_atlas_prefix=None,
)

print(f"processes in fileset: {list(fileset.keys())}")
print(f"\nexample of information in fileset:\n{{\n  'files': [{fileset['ttbar__nominal']['files'][0]}, ...],")
print(f"  'metadata': {fileset['ttbar__scaleup']['metadata']}\n}}")

processes in fileset: ['ttbar__nominal', 'ttbar__scaledown', 'ttbar__scaleup', 'ttbar__ME_var', 'ttbar__PS_var', 'single_top_s_chan__nominal', 'single_top_t_chan__nominal', 'single_top_tW__nominal', 'wjets__nominal']

example of information in fileset:
{
  'files': [https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19980_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1_00000_0000.root, ...],
  'metadata': {'process': 'ttbar', 'variation': 'scaleup', 'nevts': 12554872, 'xsec': 729.84}
}


In [53]:
NanoAODSchema.warn_missing_crossrefs = False # silences warnings about branches we will not use here

executor = processor.FuturesExecutor(workers=4)

run = processor.Runner(
    executor=executor,
    schema=NanoAODSchema,
    savemetrics=True,
    metadata_cache={},
    chunksize=200000)

treename = "Events"


filemeta = run.preprocess(fileset, treename=treename)  # pre-processing



Output()

In [54]:
labels_dict = {"ttbar": 1,
               "single_top_s_chan":2,
              "single_top_t_chan":3,
              "single_top_tW":4,
              "wjets":5}




In [55]:
# function to create column accumulator from list
def col_accumulator(a):
    return processor.column_accumulator(np.array(a))

processor_base = processor.ProcessorABC
class NSBI_analysis(processor_base):
    def __init__(self):
        super().__init__()
    
    def process(self, events):
        
        # Note: This creates new objects, distinct from those in the 'events' object
        elecs = events.Electron
        muons = events.Muon
        jets = events.Jet
        
        process = events.metadata["process"]  # "ttbar" etc.
        variation = events.metadata["variation"]  # "nominal" etc.
        
        # normalization for MC
        x_sec = events.metadata["xsec"]
        nevts_total = events.metadata["nevts"]
        lumi = 3378 # /pb

        if process != "data":
            xsec_weight = x_sec * lumi / nevts_total
        else:
            xsec_weight = 1

        electron_reqs = (elecs.pt > 30) & (np.abs(elecs.eta) < 2.1) & (elecs.cutBased == 4) & (elecs.sip3d < 4)
        muon_reqs = ((muons.pt > 30) & (np.abs(muons.eta) < 2.1) & (muons.tightId) & (muons.sip3d < 4) &
                     (muons.pfRelIso04_all < 0.15))
        jet_reqs = (jets.pt > 30) & (np.abs(jets.eta) < 2.4) & (jets.isTightLeptonVeto)

        # Only keep objects that pass our requirements
        elecs = elecs[electron_reqs]
        muons = muons[muon_reqs]
        jets = jets[jet_reqs]

        B_TAG_THRESHOLD = 0.5

        ######### Store boolean masks with PackedSelection ##########
        selections = PackedSelection(dtype='uint64')
        # Basic selection criteria
        selections.add("exactly_1l", (ak.num(elecs) + ak.num(muons)) == 1)
        selections.add("atleast_4j", ak.num(jets) >= 4)
        selections.add("atleast_1bj", ak.sum(jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) >= 1)

        selections.add("Inclusive", selections.all("exactly_1l", "atleast_4j", "atleast_1bj"))

        selection = selections.all("Inclusive")
        selected_jets = jets[selection]
        selected_elecs = elecs[selection]
        selected_muons = muons[selection]
        selected_weights = np.ones(len(selected_jets)) * xsec_weight


        # calculate number of jets in each event
        njet = ak.num(selected_jets).to_numpy()
        
        permutations_dict = get_permutations_dict(4)

        # don't consider every jet for events with high jet multiplicity
        njet[njet > max(permutations_dict.keys())] = max(permutations_dict.keys())
        # create awkward array of permutation indices
        perms = ak.Array([permutations_dict[n] for n in njet])
        perm_counts = ak.num(perms)
        

        # grab lepton info
        leptons = ak.flatten(ak.concatenate((selected_elecs, selected_muons), axis=1), axis=-1)

        H_T = ak.sum(selected_jets.pt, axis=-1)
        pT_lep = leptons.pt
        
        #### calculate features ####
        features = np.zeros((len(pT_lep), 13))
        
        features[:, 0] = pT_lep
        
        
        # pt of every jet
        features[:, 1] = ak.Array(selected_jets[:,0].pt).to_numpy()
        features[:, 2] = ak.Array(selected_jets[:,1].pt).to_numpy()
        features[:, 3] = ak.Array(selected_jets[:,2].pt).to_numpy()
        features[:, 4] = ak.Array(selected_jets[:,3].pt).to_numpy()

        # btagCSVV2 of every jet
        features[:, 5] = ak.Array(selected_jets[:,0].btagCSVV2).to_numpy()
        features[:, 6] = ak.Array(selected_jets[:,1].btagCSVV2).to_numpy()
        features[:, 7] = ak.Array(selected_jets[:,2].btagCSVV2).to_numpy()
        features[:, 8] = ak.Array(selected_jets[:,3].btagCSVV2).to_numpy()

        # quark-gluon likelihood discriminator of every jet
        features[:, 9] = ak.Array(selected_jets[:,0].qgl).to_numpy()
        features[:, 10] = ak.Array(selected_jets[:,1].qgl).to_numpy()
        features[:, 11] = ak.Array(selected_jets[:,2].qgl).to_numpy()
        features[:, 12] = ak.Array(selected_jets[:,3].qgl).to_numpy()


        train_labels = np.full_like(pT_lep, labels_dict[process])


        output = {"train_labels": col_accumulator(train_labels.tolist()),
                  "weights": col_accumulator(selected_weights.tolist()),
                  "features": col_accumulator(features.tolist()),
                    "H_T": col_accumulator(H_T.tolist()),
                    "pT_lep": col_accumulator(pT_lep.tolist()),}


        return output
        
    def postprocess(self, accumulator):
        return accumulator

In [56]:
t0 = time.monotonic()
# processing
output, metrics = run(
    fileset,
    treename,
    processor_instance=NSBI_analysis()
)


exec_time = time.monotonic() - t0


print(f"\nexecution took {exec_time:.2f} seconds")

Output()


execution took 1856.76 seconds


In [57]:
# grab features and labels and convert to np array
features = np.array(output['features'].value)
train_labels = np.array(output['train_labels'].value)
weights = np.array(output['weights'].value)

In [58]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
evaluate_only=True

In [ ]:
# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False, categories='auto')
train_labels_reshaped = train_labels.reshape(-1, 1)
train_labels_onehot = encoder.fit_transform(train_labels_reshaped)

# Standardize the input features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)  # Fit & transform training data

# Split data into training and validation sets (including weights)
X_train, X_val, y_train, y_val, weight_train, weight_val = train_test_split(
    features_scaled, train_labels_onehot, weights, test_size=0.2, random_state=42
)

# Define the neural network model
model = keras.Sequential([
    layers.Input(shape=(features.shape[1],)),  # Input layer
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(5, activation='softmax')  # Output layer for 5 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.CategoricalAccuracy(name="weighted_accuracy")])

# Train the model with sample weights
model.fit(X_train, y_train, sample_weight=weight_train, 
          validation_data=(X_val, y_val), epochs=20, batch_size=32)

# Evaluate the model using sample weights
loss, accuracy = model.evaluate(X_val, y_val, sample_weight=weight_val)
print(f"Weighted Validation Accuracy: {accuracy:.4f}")

Epoch 1/20
51188/51188 [==============================] - 40s 774us/step - loss: 0.0337 - weighted_accuracy: 0.9688 - val_loss: 0.1964 - val_weighted_accuracy: 0.9691
Epoch 2/20
51188/51188 [==============================] - 39s 764us/step - loss: 0.0329 - weighted_accuracy: 0.9687 - val_loss: 0.1927 - val_weighted_accuracy: 0.9691
Epoch 3/20
51188/51188 [==============================] - 39s 759us/step - loss: 0.0327 - weighted_accuracy: 0.9687 - val_loss: 0.1931 - val_weighted_accuracy: 0.9691
Epoch 4/20
51188/51188 [==============================] - 39s 762us/step - loss: 0.0328 - weighted_accuracy: 0.9688 - val_loss: 0.2109 - val_weighted_accuracy: 0.9691
Epoch 5/20
51188/51188 [==============================] - 39s 763us/step - loss: 0.0328 - weighted_accuracy: 0.9688 - val_loss: 0.1954 - val_weighted_accuracy: 0.9691
Epoch 6/20
51188/51188 [==============================] - 39s 760us/step - loss: 0.0326 - weighted_accuracy: 0.9688 - val_loss: 0.2021 - val_weighted_accuracy: 0.969

In [ ]:
print(y_val)
true_labels = np.argmax(y_val, axis=1) + 1  # Convert from one-hot to 1-based class labels
print(true_labels[true_labels==5].shape)


In [ ]:
# Get predictions (softmax outputs)
y_pred = model.predict(X_val)  # Shape: (num_samples, 5)

# Convert one-hot encoded y_val back to integer labels (1-5)
true_labels = np.argmax(y_val, axis=1) + 1  # Converts from one-hot to 1-5 based labels


In [ ]:
for key in labels_dict:  # Loop over 5 classes
    plt.hist(y_pred[true_labels == labels_dict[key]], bins=50, 
             alpha=0.6, label=key, 
             density=True)

plt.xlabel("Softmax Output")
plt.ylabel("Density")
plt.title("Distribution of Neural Network Softmax Outputs per Class")
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
for key in labels_dict:  # Loop over 5 classes
    plt.hist(y_pred[true_labels == labels_dict[key]], bins=50, 
             alpha=0.6, label=key, 
             density=True)

plt.xlabel("Softmax Output")
plt.ylabel("Density")
plt.title("Distribution of Neural Network Softmax Outputs per Class")
plt.legend()
plt.yscale('log')
plt.show()